In [ ]:
import requests
indexes = requests.get('http://localhost:9200/_cat/indices',auth=('elastic', 'changeme'))
print(indexes.text)

In [133]:
import bs4
import json
import requests
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
es = Elasticsearch([{'host': 'localhost', 'port': 9200}],http_auth=('elastic', 'changeme'),)

In [134]:
#PUT /bible
index = '{ \
    "mappings": { \
        "verse": { \
            "properties": { \
                "epochtime_loaded": { "type": "date" }, \
                "version": { \
                    "type": "keyword", \
                    "index": false \
                }, \
                "book": { \
                    "type": "keyword", \
                    "index": false \
                }, \
                "chapter_no": {"type": "integer"}, \
                "verse_no": {"type": "integer"}, \
                "verse" : {"type": "text", \
                  "index": true \
                }, \
                "es_id" : {"type": "text","index": false} \
            } \
        } \
    } \
}'

In [ ]:
json_data = {
    "query" :{
        "match_all" : {}
    }
}

res = es.search(index=".watches", body=json_data)
print(json.dumps(res["hits"], indent=2))

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("/Users/jeff/Downloads/bibles.txt", sep='\t', header=0 )

In [ ]:
from IPython.display import display, HTML

# Assuming that dataframes df1 and df2 are already defined:
#print "Dataframe 1:"
#bcv = df.Verse.str.split(' ')
#print bcv[0][1]
#cv = str(bcv[0][1]).split(':')
#print cv
#display(df.Verse.str.split(' '))
print "Dataframe 2:"
HTML(df.to_html(max_rows=10))

In [11]:
import time
import re

def createNewJSON (row, version, x_id):
    p = re.compile('\d+:\d+')
    m  = p.search(row.Verse)
    book = row.Verse[:m.start()-1]
    cv = str(m.group()).split(':')
    ps = {}
    ps['type'] = 'verse'
    ps['line_id'] = str(x_id)
    ps['book'] = book
    ps['chapter_no'] = str(cv[0])
    ps['verse_no'] = str(cv[1])
    ps['text_entry'] = row[version]
    #ps['es_id'] = bv['code'] + '_' + str(row.Verse).replace(" ", "_")
    #ps['epochtime_loaded'] = time()
    return ps

In [31]:
def writeFile (data):
    with open('/Users/jeff/jupyter/' + index + '-6.0.json', 'a') as outfile:
        outfile.write(data)

In [32]:
import json
x_id = 0
#print index_str
for index, row in df.iterrows():
    version = "American King James Version"
    index = 'american_king_james'
    index_str = '{"index":{"_index":"' + index + '","_id":' + str(x_id) + '}}'
    ps = createNewJSON(row,version,x_id)
    writeFile(index_str + '\n' + str(json.dumps(ps)) + '\n',index)
    x_id = x_id + 1

In [34]:
versions = [['American King James Version', 'american_king_james'],['American Standard','american_standard'], "Young's Literal Translation"]

In [38]:
print versions[0]

American King James Version
